In [ ]:
import pynq
from pynq.mmio import MMIO
from pynq.lib.video import *

import numpy as np

import cv2

import time

import PIL.Image

from stewart_controller import Stewart_Platform

import math

import collections

from IPython.display import clear_output

from stewart_controller import Stewart_Platform
from ipywidgets import interact, interactive, widgets, fixed
import numpy as np

from const import *
from servo import map_servo_pwm_signal, radian_to_pwm
from pipelineOverlay import PipelineOverlay

In [2]:
# Load Base Overlay
base = PipelineOverlay("/home/xilinx/jupyter_notebooks/bit_files/top_level.bit")

In [3]:
# Array for storing the last set servo values
servo_values = [0 for x in range(0,6)]

# Servo Calibration

This script generates one slider for each servo. Check if each number corresponds to the correct output of the board. If the servos are not in sequential order on the platform, then you need to set `servo_reorder_map` in the `servo.py`. This will not effect this script but is needed for the platform later, because the reverse Kinematic will return a list of angles which is ordered for a sequential order of servos around the platform.

To calibrate the servos use the sliders to point their arms to -90° and +90° and note the values at these points. The arm is at 0° when it is laying flat in the x direction. Write these values into the `servo_limits` array in `servo.py`. 

In [ ]:
from ipywidgets import interact, interactive, widgets, fixed
import numpy as np

radian = False

# 0 = 610, 2530
# 1 = 460, 2390
# 2 = 540, 2390
# 3 = 600, 2560
# 4 = 380, 2320
# 5 = 480, 2400
result = [
    [540, 2390],
    [600, 2560],
    [610, 2530],
    [460, 2390],
    [480, 2400],
    [380, 2320],
]

def set_servo(set_point, servo):
    """
        This function sets a single servo position. 
    """
    if radian:
        pwm = radian_to_pwm(set_point, servo, result)
    else:
        pwm = int(set_point)
    
    # pwm = map_servo_pwm_signal(servo, pwm, result)
    
    base.pwm_controller_set(servo, pwm) 
    base.pwm_controller_commit()
    servo_values[servo] = pwm
    return pwm

for i in range(0, 6):
    min_value = 0
    max_value = 6000
    step = 10
    value = servo_values[i]
    if radian:
        min_value = -np.pi/2
        max_value = np.pi/2
        step = 0.1
        value = 0
        
    interact(
        set_servo,
        servo=fixed(i),
        set_point=widgets.FloatSlider(
            min=min_value,
            max=max_value,
            value=value,
            step=step,
            description=f"Servo {i}",
            continuous_update=True,
        )
    )